In [1]:
import matplotlib.pyplot as plt
import cv2
import xml.etree.ElementTree as ET
import os
from shapely.geometry import Polygon
import keras
import numpy as np

In [2]:
def getAnnotationIndex(prediction_box, annotation_boxes):
    max_overlap = -1
    max_overlap_index = -1
    for i in range(len(annotation_boxes)):
        overlap = calculate_iou(transformBox(prediction_box), transformBox(annotation_boxes[i]))
        if overlap > max_overlap:
            max_overlap = overlap
            max_overlap_index = i
    return max_overlap_index

def transformBox(box):
    x1 = box[0]
    y1 = box[1]
    
    x2 = box[2]
    y2 = box[3]
    
    return [[x1, y1], [x2, y1], [x2,y2], [x1,y2]]

def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
    return iou

def parse_annot(xml):
    bbox = []
    labels = []
    
    label_map = {
        'without_mask': 1,
        'with_mask': 2,
        'mask_weared_incorrect': 3 
    }
    
    tree = ET.parse(xml)
    root = tree.getroot()
    
    for box in root.iter('object'):
        bbox.append(
               [
                   int(box.find('bndbox/xmin').text),
                   int(box.find('bndbox/ymin').text),
                   int(box.find('bndbox/xmax').text),
                   int(box.find('bndbox/ymax').text),
               ]
        )
        labels.append(
            label_map[box.find('name').text]
        )
    
    return bbox, labels

In [6]:
# FILE PATHS
IMAGE_PATH = 'data/images'
XML_PATH = 'data/annotations'

In [7]:
# FILES SORTED
images = list(sorted(filter(lambda x: os.path.splitext(x)[1] == '.png', os.listdir(IMAGE_PATH))))
xmls = list(sorted(filter(lambda x: os.path.splitext(x)[1] == '.xml', os.listdir(XML_PATH))))

In [8]:
# HAAR CASCADE MODEL
face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [9]:
# PREDICTING IMAGES
prediction_boxes = []
for image in images:
    img = cv2.imread(os.path.join(IMAGE_PATH, image))
    img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
    pred_boxes = face_model.detectMultiScale(img,scaleFactor=1.1, minNeighbors=4)
    pred_boxes_rect = []
    for (x,y,w,h) in pred_boxes:
        pred_boxes_rect.append([x,y,x+w,y+h])
    prediction_boxes.append(pred_boxes_rect)

In [10]:
# GROUND TRUTH EXTRACTED FROM ANNOTATIONS
annot_boxes = []
annot_labels = []
annot_paths = []
for xml in xmls:
    bxs, lbls = parse_annot(os.path.join(XML_PATH, xml))
    annot_boxes.append(bxs)
    annot_labels.append(lbls)
    annot_paths.append(xml)

In [11]:
# PREDICTION OF THE FIRST IMAGE
print(prediction_boxes[0])
# GROUND TRUTH OF THE FIRST IMAGE
print(annot_boxes[0])

[[310, 82, 366, 138], [9, 160, 68, 219]]
[[79, 105, 109, 142], [185, 100, 226, 144], [325, 90, 360, 141]]


In [12]:
# CALCULATING BOX OVERLAP WITH ANNOTATIONS
overall_boxes = 0
overlap = 0
for i in range(len(prediction_boxes)):
    for pred_box in prediction_boxes[i]:
        j = getAnnotationIndex(pred_box, annot_boxes[i]) # get the closest annotation box
        overlap += calculate_iou(transformBox(pred_box), transformBox(annot_boxes[i][j]))
    overall_boxes += len(prediction_boxes[i])
print("Overlap: " + str(overlap / overall_boxes))

Overlap: 0.491874487520049


In [10]:
# CALCULATING LABELS OVERLAP - MASK ON/OFF DETECTION BY ADDITIONAL VGG19 MODEL
load_model = keras.models.load_model('masknet.h5')
labels_dict={0:'with mask',1:'without mask'}
color_dict={0:(0,255,0),1:(0,0,255)}
correct = 0
overall_labels = 0
incorrect_detections = []
for i in range(len(prediction_boxes)):
    img = cv2.imread(os.path.join(IMAGE_PATH, images[i]))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    for pred_box in prediction_boxes[i]:
        x = pred_box[0]
        y = pred_box[1]
        h = pred_box[2] - pred_box[0]
        w = pred_box[3] - pred_box[1]
        crop = img[y:y+h,x:x+w]
        crop = cv2.resize(crop,(128,128))
        crop = np.reshape(crop,[1,128,128,3])/255.0
        result=load_model.predict(crop)
        if result[0][0] > result[0][1]:
            percent = round(result[0][0]*100,2)
        else:
            percent = round(result[0][1]*100,2)
        face_label=np.argmax(result,axis=1)[0]
        j = getAnnotationIndex(pred_box, annot_boxes[i])
        if face_label == annot_labels[i][j]:
            correct += 1
        overall_labels +=1
print("Label overlap: " + str(correct) + "/" + str(overall_labels))
print("Label prediction accuracy: " + str(correct / overall_labels))

Label overlap: 145/716
Label prediction accuracy: 0.20251396648044692
